In [1]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install kafka-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys
import random
import time
import json
from datetime import datetime
from kafka import KafkaProducer
from kafka.errors import NoBrokersAvailable
import psycopg2

# 🔌 اتصال بـ Kafka
attempt = 1
while True:
    try:
        print(f"🔌 Attempt {attempt}: Connecting to Kafka...")
        producer = KafkaProducer(
            bootstrap_servers='broker:29092',  # Docker network
            value_serializer=lambda v: json.dumps(v).encode('utf-8')
        )
        print("✅ Connected to Kafka!")
        break
    except NoBrokersAvailable:
        print("❌ Kafka not available yet, retrying in 5 seconds...")
        time.sleep(5)
        attempt += 1

# 🔌 اتصال بـ PostgreSQL
conn = psycopg2.connect(
    host="postgres_general",
    database="patients_db",
    user="admin",
    password="admin"
)
cursor = conn.cursor()
print("✅ Connected to PostgreSQL!")

patients = {}

def fluctuate(value, min_val, max_val, step=0.5):
    change = round(random.uniform(-step, step), 1)
    new_value = value + change
    return max(min_val, min(new_value, max_val))

def update_patient_data(patient_id, device_id):
    if patient_id not in patients:
        patients[patient_id] = {
            "heart_rate": random.randint(70, 90),
            "oxygen_saturation": round(random.uniform(95, 99), 1),
            "temperature": round(random.uniform(36.5, 37.5), 1),
            "blood_pressure": f"{random.randint(110, 130)}/{random.randint(70, 85)}",
            "respiratory_rate": random.randint(12, 20),
            "glucose_level": round(random.uniform(80, 120), 1),
            "status": "stable",
            "critical_cycles": 0
        }

    patient = patients[patient_id]

    # تحديث القيم حسب الحالة
    if patient["status"] == "critical":
        patient["heart_rate"] = fluctuate(patient["heart_rate"], 100, 110)
        patient["oxygen_saturation"] = fluctuate(patient["oxygen_saturation"], 88, 92)
        patient["temperature"] = fluctuate(patient["temperature"], 38.5, 39.0)
        patient["respiratory_rate"] = fluctuate(patient["respiratory_rate"], 22, 30)
        patient["glucose_level"] = fluctuate(patient["glucose_level"], 150, 200)
        patient["critical_cycles"] += 1
        if patient["critical_cycles"] >= 3:
            patient["status"] = "warning"
            patient["critical_cycles"] = 0
    elif patient["status"] == "warning":
        patient["heart_rate"] = fluctuate(patient["heart_rate"], 90, 100)
        patient["oxygen_saturation"] = fluctuate(patient["oxygen_saturation"], 92, 95)
        patient["temperature"] = fluctuate(patient["temperature"], 37.5, 38.5)
        patient["respiratory_rate"] = fluctuate(patient["respiratory_rate"], 18, 22)
        patient["glucose_level"] = fluctuate(patient["glucose_level"], 120, 150)
        if random.random() < 0.3:
            patient["status"] = "stable"
        elif random.random() < 0.2:
            patient["status"] = "critical"
    else:
        patient["heart_rate"] = fluctuate(patient["heart_rate"], 70, 90)
        patient["oxygen_saturation"] = fluctuate(patient["oxygen_saturation"], 95, 99)
        patient["temperature"] = fluctuate(patient["temperature"], 36.5, 37.5)
        patient["respiratory_rate"] = fluctuate(patient["respiratory_rate"], 12, 20)
        patient["glucose_level"] = fluctuate(patient["glucose_level"], 80, 120)
        if random.random() < 0.1:
            patient["status"] = "warning"

    systolic = random.randint(100, 140)
    diastolic = random.randint(60, 90)
    patient["blood_pressure"] = f"{systolic}/{diastolic}"
    alert = patient["status"] == "critical"

    return {
        "patient_id": patient_id,
        "device_id": device_id,
        "heart_rate": round(patient["heart_rate"], 1),
        "blood_pressure": patient["blood_pressure"],
        "oxygen_saturation": round(patient["oxygen_saturation"], 1),
        "temperature": round(patient["temperature"], 1),
        "respiratory_rate": round(patient["respiratory_rate"], 1),
        "glucose_level": round(patient["glucose_level"], 1),
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "status": patient["status"],
        "alert": alert
    }

NUM_PATIENTS = 10

while True:
    print("🔄 Starting new cycle...")
    for i in range(1, NUM_PATIENTS + 1):
        patient_id = f"P{i:03}"
        device_id = f"DEV{i:03}"
        data = update_patient_data(patient_id, device_id)

        # 1️⃣ إرسال البيانات للـ Kafka
        producer.send('sensordata', value=data)
        print(f"📤 Sent to Kafka: {data}")

        # 2️⃣ إدخال البيانات في PostgreSQL
        insert_query = """
        INSERT INTO vitals (
            patient_id, device_id, heart_rate, blood_pressure,
            oxygen_saturation, temperature, respiratory_rate, glucose_level,
            timestamp, status, alert
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (
            data['patient_id'], data['device_id'], data['heart_rate'], data['blood_pressure'],
            data['oxygen_saturation'], data['temperature'], data['respiratory_rate'], data['glucose_level'],
            data['timestamp'], data['status'], data['alert']
        ))
        conn.commit()

    print("⏳ Waiting 50 seconds...\n")
    time.sleep(50)


🔌 Attempt 1: Connecting to Kafka...
✅ Connected to Kafka!
✅ Connected to PostgreSQL!
🔄 Starting new cycle...
📤 Sent to Kafka: {'patient_id': 'P001', 'device_id': 'DEV001', 'heart_rate': 80.0, 'blood_pressure': '117/85', 'oxygen_saturation': 96.3, 'temperature': 37.3, 'respiratory_rate': 14.3, 'glucose_level': 85.9, 'timestamp': '2025-11-21 12:07:08', 'status': 'stable', 'alert': False}
📤 Sent to Kafka: {'patient_id': 'P002', 'device_id': 'DEV002', 'heart_rate': 76.1, 'blood_pressure': '125/63', 'oxygen_saturation': 95.9, 'temperature': 37.1, 'respiratory_rate': 15.6, 'glucose_level': 91.1, 'timestamp': '2025-11-21 12:07:08', 'status': 'stable', 'alert': False}
📤 Sent to Kafka: {'patient_id': 'P003', 'device_id': 'DEV003', 'heart_rate': 82.5, 'blood_pressure': '102/64', 'oxygen_saturation': 99, 'temperature': 37.2, 'respiratory_rate': 15.1, 'glucose_level': 112.2, 'timestamp': '2025-11-21 12:07:08', 'status': 'stable', 'alert': False}
📤 Sent to Kafka: {'patient_id': 'P004', 'device_id':

KeyboardInterrupt: 